# Overview

This examples uses symbolic execution to solve the challenge binary.

Compile the binary with `gcc challenge.c -no-pie -o challenge`

This example embeds and NP-complete problem ([integer factorization](https://en.wikipedia.org/wiki/Factorization#Integers)) into the the analyis challenge. This is not meant to be a critique of symbolic execution in general, since truly any program analysis technique will struggle with this example! In fact, if there were an efficient automatic technique to solve this challenge we would have bigger problems in security (our cypto algorithms rely on this problem being hard to solve).

## Experiment:
Try a few experiements and observe how it affects the solver. Does the solver produce a correct result? The solver produce a timely result?
- Change the source to fix `a` or `b` and instead only read one value?
- Try smaller vs. larger primes (2,5) vs (104639,104729)
- Try changing the size of types used to compute the branch condition (int vs long long)

In [83]:
program_directory = "/home/pac/Desktop/labs/lab7/challenge/"
program_path = program_directory + "challenge"

In [84]:
import os
print(os.popen("gdb " + program_path + " -batch -ex 'disassemble main'").read())

Dump of assembler code for function main:
   0x0000000000400632 <+0>:	push   rbp
   0x0000000000400633 <+1>:	mov    rbp,rsp
   0x0000000000400636 <+4>:	sub    rsp,0x20
   0x000000000040063a <+8>:	mov    DWORD PTR [rbp-0x14],edi
   0x000000000040063d <+11>:	mov    QWORD PTR [rbp-0x20],rsi
   0x0000000000400641 <+15>:	mov    rax,QWORD PTR fs:0x28
   0x000000000040064a <+24>:	mov    QWORD PTR [rbp-0x8],rax
   0x000000000040064e <+28>:	xor    eax,eax
   0x0000000000400650 <+30>:	lea    rax,[rbp-0x10]
   0x0000000000400654 <+34>:	mov    rsi,rax
   0x0000000000400657 <+37>:	lea    rdi,[rip+0x134]        # 0x400792
   0x000000000040065e <+44>:	mov    eax,0x0
   0x0000000000400663 <+49>:	call   0x400510 <__isoc99_scanf@plt>
   0x0000000000400668 <+54>:	lea    rax,[rbp-0xc]
   0x000000000040066c <+58>:	mov    rsi,rax
   0x000000000040066f <+61>:	lea    rdi,[rip+0x11c]        # 0x400792
   0x0000000000400676 <+68>:	mov    eax,0x0
   0x000000000040067b <+73>:	call   0x400510 <__isoc99_scanf@plt>


In [85]:
# import the python system and angr libraries

import angr
import sys

In [86]:
# load the challenge binary
proj = angr.Project(program_path)

main_address = proj.loader.find_symbol("main").rebased_addr

# recovered through objdump, gdb, radare2, or some dissassembler the offset
# to the start of the basic block that calls the print_flag function
target_address_offset = 139 # start of basic block with call to print_flag
target_address = main_address + target_address_offset

rejected_address_offset = 151
rejected_address = main_address + rejected_address_offset

In [87]:
# start the program state at the beginning of main
state = proj.factory.entry_state(addr=main_address)
sm = proj.factory.simulation_manager(state)

In [ ]:
# symbolically explore the program, solving for inputs that satisfy paths to the target address
sm.explore(find=target_address, avoid=[rejected_address])
while len(sm.found) == 0:
    sm.step()

# if the solver found a way to reach the target address then print the input
if (len(sm.found) > 0):
    print("Found an input to satisfy the path constraints to reach target address!")
    found_input = sm.found[0].posix.dumps(sys.stdin.fileno())
    print(found_input)
    with open("analysis_result", "wb") as fp:
        fp.write(found_input)
else:
    print("Unable to compute an input that satisfies the path constraints to reach the target address")

WARNING | 2020-05-25 20:29:30,387 | angr.state_plugins.symbolic_memory | Filling register rdi with 8 unconstrained bytes referenced from 0x40063a (main+0x8 in challenge (0x40063a))
WARNING | 2020-05-25 20:29:30,414 | angr.state_plugins.symbolic_memory | Filling register rsi with 8 unconstrained bytes referenced from 0x40063d (main+0xb in challenge (0x40063d))


In [ ]:
# test the computed input on the challenge program

command = program_path + " < " + program_directory + "analysis_result"
print(os.popen(command).read())